# Music Genre Recognition - Milestone 1
Darren Midkiff and Cheng-Wei Hu

## Overview
This project aims to identify the genre of music in an audio sample. Features will be extracted from the analog data, and a genre will be predicted using a KNN model, trained on labelled audio samples from the freely available FMA dataset.

## Training Data

The [FMA Dataset](https://github.com/mdeff/fma) is comprised of over 100,000 tracks from 161 genres. In order to make the problem more manageable, we will use the small version of the dataset, which includes 8,000 tracks from 8 top-level genres. The dataset also includes dozens of features -- year released, location of artist, number of listens, etc. Because this project aims to identify genre using only audio signal, all of these features are irrelevant and will be dropped.

In [7]:
import pandas as pd

#read full metadata file
metadata = pd.read_csv("./fma_metadata/tracks.csv", skiprows=[0,2], low_memory=False)

# drop all tracks that are not in fma_small dataset
metadata = metadata[metadata["subset"].eq("small")]
# add name to track_id column (missing because of stupid CSV formatting)
metadata = metadata.rename(columns={"Unnamed: 0": "track_id"})
# drop all columns that don't relate to genre
# we will not have this metadata from the audio file
metadata.drop(metadata.columns.difference(["track_id","genre_top"]),1,inplace=True)
# reset indices accounting for dropped rows
metadata = metadata.reset_index(drop=True)

# #write only relevant metadata to file for use in training
# metadata.to_csv("fma_small_genres.csv")

metadata.head()

,track_id,genre_top
0,2,Hip-Hop
1,5,Hip-Hop
2,10,Pop
3,140,Folk
4,141,Folk


In [9]:
print(metadata["genre_top"].unique())

['Hip-Hop' 'Pop' 'Folk' 'Experimental' 'Rock' 'International' 'Electronic'
 'Instrumental']


## Feature Extraction

## Model